In [1]:
import torch
import torch.nn as nn

In [2]:
with open("Ok.txt" , "r" ,encoding='utf-8') as f:
    text = f.read()
print(len(text))

250546


In [3]:
chars = sorted(set(text))
print(chars)

['\n', ' ', '!', '"', '$', '%', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '‘', '’', '“', '”', '•', '™', '\ufeff']


In [4]:
string_to_int = {ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }

In [5]:
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: "".join([int_to_string[i] for i in l])

In [6]:
num = encode(text)
num[:20]

[90, 47, 71, 1, 40, 81, 1, 45, 61, 57, 60, 61, 74, 75, 0, 0, 0, 36, 76, 7]

In [7]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4

cuda


In [9]:
tensor_data = torch.tensor(encode(text),dtype=torch.long)
print(tensor_data.shape)

torch.Size([250546])


In [10]:
n = int(0.8*len(tensor_data))
train_data = tensor_data[:n]
val_data = tensor_data[n:]

In [11]:
block_size = 8 
x = train_data[:block_size]
y = train_data[1:block_size+1]

for i in range(block_size):
    context = x[:i+1]
    target = y[i]
    print("when input is",context , "target is",target)

when input is tensor([90]) target is tensor(47)
when input is tensor([90, 47]) target is tensor(71)
when input is tensor([90, 47, 71]) target is tensor(1)
when input is tensor([90, 47, 71,  1]) target is tensor(40)
when input is tensor([90, 47, 71,  1, 40]) target is tensor(81)
when input is tensor([90, 47, 71,  1, 40, 81]) target is tensor(1)
when input is tensor([90, 47, 71,  1, 40, 81,  1]) target is tensor(45)
when input is tensor([90, 47, 71,  1, 40, 81,  1, 45]) target is tensor(61)


250546

In [20]:
n = int(0.8*len(tensor_data))
block_size = 8
batch_size = 4
train_data = tensor_data[:n]
val_data = tensor_data[n:]

def get_batch(split):
    if split.lower()=='train':
        data = train_data
    else:
        data = val_data
    ix = torch.randint(len(data)-block_size,(batch_size,))
    print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([
        data[i+1:i+block_size+1] for i in ix])
    x,y = x.to(device) , y.to(device)
    return x,y


x,y = get_batch('test')

print('inputs')
print(x)

print('targets')
print(y)

tensor([35731,  2594, 26877, 47146])
inputs
tensor([[39, 65, 59, 61, 70, 75, 61,  1],
        [67, 75,  1, 71, 62,  1, 42, 82],
        [ 1, 64, 61,  1, 64, 57, 60,  1],
        [75, 13,  0,  0,  0, 47, 64, 61]], device='cuda:0')
targets
tensor([[65, 59, 61, 70, 75, 61,  1, 79],
        [75,  1, 71, 62,  1, 42, 82,  1],
        [64, 61,  1, 64, 57, 60,  1, 75],
        [13,  0,  0,  0, 47, 64, 61,  1]], device='cuda:0')


In [12]:
class Model(nn.Module):
    def __init__(self,vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)
        print("sucess")
    def forward(self,index,targets):
        logits = self.token_embedding_table(index)
        return logits